# Multi-Step DQN — Rezumat

**Multi-Step DQN** extinde algoritmul DQN prin folosirea unui *n-step return*,
adică ia în considerare mai multe recompense viitoare înainte de a actualiza
valoarea unei acțiuni.

În loc de actualizarea clasică pe un singur pas:

$$
y = r + \gamma \, Q(s', a')
$$

Multi-Step folosește:

$$
y^{(n)} = 
\sum_{k=0}^{n-1} \gamma^{k} r_{k}
\;+\;
\gamma^{n} Q(s_{t+n}, a_{t+n})
$$

---

## 🎯 De ce Multi-Step?

- oferă un **semnal de învățare mai bogat**  
- reduce dependența de targeturile instabile one-step  
- antrenează agentul să învețe **lanțuri întregi de consecințe**, nu doar pasul următor  
- convergență mai rapidă și comportament mai stabil

---

## 🧠 Cum funcționează?

1. Stocăm tranzițiile într-un buffer temporar de **n pași**.  
2. Când se strâng `n` tranziții, calculăm suma discountată a recompenselor.  
3. Învățăm din primul stat → și ultimul stat din fereastra de n pași.  
4. Aplicăm update-ul Double DQN normal, dar cu targetul multi-step.

---

## ⭐ Avantaje

- învață secvențe, nu doar reacții imediate  
- mult mai eficient în medii cu recompense întârziate  
- este parte esențială din **Rainbow DQN**

---

## 📌 Pe scurt

**Multi-Step DQN = DQN + update pe n pași cu discount acumulat**,  
ce oferă un semnal mai puternic și accelerează învățarea.

Este o îmbunătățire simplă, dar foarte importantă, folosită în toate DQN-urile moderne.


In [ ]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
import random
import collections
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow.keras import Model, layers, optimizers


# ============================================================
# Multi-Step Replay Buffer (n-step returns)
# ============================================================

class MultiStepReplayBuffer:
    def __init__(self, capacity, state_dim, n_step=3, gamma=0.99):
        self.capacity = capacity
        self.ptr = 0
        self.full = False

        self.states = np.zeros((capacity, state_dim), dtype=np.float32)
        self.next_states = np.zeros((capacity, state_dim), dtype=np.float32)
        self.actions = np.zeros(capacity, dtype=np.int32)
        self.rewards = np.zeros(capacity, dtype=np.float32)
        self.dones = np.zeros(capacity, dtype=np.float32)

        self.n_step = n_step
        self.gamma = gamma
        self.n_step_buffer = collections.deque(maxlen=n_step)

    def store(self, s, s2, a, r, d):
        # Add transition to n-step buffer
        self.n_step_buffer.append((s, s2, a, r, d))

        # Wait until we have full n-step window
        if len(self.n_step_buffer) < self.n_step:
            return

        # Compute n-step return
        R = 0
        discount = 1
        for (_, _, _, reward_i, done_i) in self.n_step_buffer:
            R += reward_i * discount
            discount *= self.gamma
            if done_i:
                break

        # First transition in buffer is the one that gets recorded
        s_0, _, a_0, _, _ = self.n_step_buffer[0]
        _, s_n, _, _, d_n = self.n_step_buffer[-1]

        idx = self.ptr
        self.states[idx] = s_0
        self.next_states[idx] = s_n
        self.actions[idx] = a_0
        self.rewards[idx] = R
        self.dones[idx] = float(d_n)

        self.ptr = (self.ptr + 1) % self.capacity
        if self.ptr == 0:
            self.full = True

    def sample(self, batch_size):
        max_size = self.capacity if self.full else self.ptr
        idx = np.random.choice(max_size, batch_size, replace=False)

        return (
            self.states[idx],
            self.next_states[idx],
            self.actions[idx],
            self.rewards[idx],
            self.dones[idx]
        )


# ============================================================
# Basic DQN Network (2 hidden layers)
# ============================================================

class QNetwork(Model):
    def __init__(self, n_actions):
        super().__init__()
        self.fc1 = layers.Dense(128, activation='relu')
        self.fc2 = layers.Dense(128, activation='relu')
        self.fc3 = layers.Dense(n_actions)

    def call(self, x):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        x = tf.nn.relu(self.fc1(x))
        x = tf.nn.relu(self.fc2(x))
        return self.fc3(x)

    def act(self, state, epsilon):
        if random.random() < epsilon:
            return random.randint(0, self.fc3.units - 1)
        state = state[np.newaxis, :]
        Q = self.call(state)[0]
        return int(tf.argmax(Q).numpy())


# ============================================================
# Hyperparameters
# ============================================================

GAMMA = 0.99
LR = 1e-3
BATCH = 32
MEM_SIZE = 50000
NUM_EPISODES = 600

N_STEP = 3               # <<< multi-step return
GAMMA_N = GAMMA ** N_STEP

INITIAL_EPS = 1.0
FINAL_EPS = 0.02
EPS_DECAY = 0.995

TAU = 0.005


# ============================================================
# Soft update function
# ============================================================

def soft_update(target, online, tau=TAU):
    tw = target.get_weights()
    ow = online.get_weights()
    new_w = [t * (1 - tau) + o * tau for t, o in zip(tw, ow)]
    target.set_weights(new_w)


# ============================================================
# Training Loop — Multi-Step DQN
# ============================================================

env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
n_actions = env.action_space.n

buffer = MultiStepReplayBuffer(MEM_SIZE, state_dim, n_step=N_STEP, gamma=GAMMA)
online = QNetwork(n_actions)
target = QNetwork(n_actions)
target.set_weights(online.get_weights())

optimizer = optimizers.Adam(LR)
mse = tf.keras.losses.MeanSquaredError()

epsilon = INITIAL_EPS
reward_history = []

for episode in range(NUM_EPISODES):

    state, _ = env.reset()
    ep_reward = 0

    for step in range(200):

        action = online.act(state, epsilon)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        ep_reward += reward

        buffer.store(state, next_state, action, reward, float(done))
        state = next_state

        # ----------------------------------------------------
        # TRAINING
        # ----------------------------------------------------
        if buffer.ptr > 1000 or buffer.full:

            states, next_states, actions, rewards, dones = buffer.sample(BATCH)

            # Double DQN: choose action with online, evaluate with target
            next_actions = tf.argmax(
                online(next_states), axis=1, output_type=tf.int32
            )
            idx = tf.stack([tf.range(BATCH, dtype=tf.int32), next_actions], axis=1)

            target_Q_values = tf.gather_nd(target(next_states), idx)

            y = rewards + (1 - dones) * GAMMA_N * target_Q_values

            with tf.GradientTape() as tape:
                q_pred = online(states)
                a_idx = tf.stack([tf.range(BATCH, dtype=tf.int32), actions], axis=1)
                selected_q = tf.gather_nd(q_pred, a_idx)
                loss = mse(selected_q, y)

            grads = tape.gradient(loss, online.trainable_variables)
            grads = [tf.clip_by_norm(g, 5.0) for g in grads]
            optimizer.apply_gradients(zip(grads, online.trainable_variables))

            soft_update(target, online)

        if done:
            break

    epsilon = max(FINAL_EPS, epsilon * EPS_DECAY)
    reward_history.append(ep_reward)

    # ----------------------------------------------------
    # LIVE PLOT
    # ----------------------------------------------------
    if episode % 5 == 0:
        clear_output(wait=True)
        plt.figure(figsize=(10,4))

        # Reward
        plt.plot(
            reward_history,
            label="Reward",
            color="blue",
            alpha=0.3,   
            linewidth=1
        )

        # Moving average
        if len(reward_history) > 20:
            ma = np.convolve(reward_history, np.ones(20)/20, mode='valid')
            plt.plot(
                range(19, len(reward_history)),
                ma,
                label="Moving Avg (20 eps)",
                color="orange",
                linewidth=2.5
            )

        plt.title("Multi-Step DQN — Training Progress")
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

